In [1]:
import tensorflow as tf
from keras import layers
from keras.datasets import mnist
from keras.layers import Dropout
from tensorflow import keras

import plotly.express as px

2022-04-08 21:44:29.720943: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-08 21:44:29.720991: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [4]:
import os, shutil, pathlib

original_dir = pathlib.Path("../dataset/intel_images")
new_base_dir = pathlib.Path("../dataset/intel_images_small")

In [30]:
def make_subset(subset_name, start_idx, end_idx):
    for category in("buildings", "forest", "glacier", "mountain", "sea", "street"):
        new_dir = new_base_dir / subset_name / category
        if not new_dir.exists():
            os.makedirs(new_dir)
        fnames = [os.listdir(original_dir / category)[i] for i in range(start_idx, end_idx)]
        for fname in fnames:
            shutil.copyfile(src=original_dir / category/ fname, dst=new_dir / fname)

In [21]:
# create train test function for splitting of intel images from kaggle
# def create_train_test_split(train_size=0.8):
#     train_dir = new_base_dir / "train"
#     test_dir = new_base_dir / "test"
#     if not train_dir.exists():
#         os.makedirs(train_dir)
#     if not test_dir.exists():
#         os.makedirs(test_dir)
#     for category in("buildings", "forest", "glacier", "mountain", "sea", "street"):
#         new_dir = new_base_dir / category
#         if not new_dir.exists():
#             os.makedirs(new_dir)
#         fnames = [f"{i}.jpg" for i in range(0, len(os.listdir(original_dir/category)))]
#         for fname in fnames:
#             shutil.copyfile(src=original_dir/ category / fname, dst=new_dir / train_dir / fname)
#         fnames = [f"{i}.jpg" for i in range(len(os.listdir(original_dir)), len(os.listdir(original_dir)))]
#         for fname in fnames:
#             shutil.copyfile(src=new_dir / fname, dst=new_dir / test_dir / fname)

Make train test and valid set

In [31]:
make_subset("train", 0, 1000)
make_subset("valid", 1000, 1500)
make_subset("test", 1500, 2000)

### Model development

In [32]:
inputs = keras.Input(shape=(180, 180, 3))
x = layers.Rescaling(1. / 255)(inputs)
x = layers.Conv2D(filters=32, kernel_size=3, activation='relu')(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation='relu')(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation='relu')(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation='relu')(x)
x = layers.Flatten()(x)

outputs = layers.Dense(units=6, activation='softmax')(x)
model = keras.Model(inputs=inputs, outputs=outputs)

2022-04-08 22:41:25.905047: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-04-08 22:41:25.905089: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-08 22:41:25.905109: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (linux): /proc/driver/nvidia/version does not exist
2022-04-08 22:41:25.905401: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [33]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 180, 180, 3)]     0         
                                                                 
 rescaling (Rescaling)       (None, 180, 180, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 178, 178, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 89, 89, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 87, 87, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 43, 43, 64)       0         
 2D)                                                         

In [34]:
model.compile(
    optimizer=keras.optimizers.Adam(lr=0.001),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
    )


/home/avizyt/anaconda3/envs/tflow/lib/python3.9/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Setting up the dataset for training

In [37]:
from tensorflow.keras.utils import image_dataset_from_directory

train_dataset = image_dataset_from_directory(
    directory=new_base_dir / "train",
    image_size=(180, 180),
    batch_size=32
)
validation_dataset = image_dataset_from_directory(
    directory=new_base_dir / "valid",
    image_size=(180, 180),
    batch_size=32
)
test_dataset = image_dataset_from_directory(
    directory=new_base_dir / "test",
    image_size=(180, 180),
    batch_size=32
)

Found 6000 files belonging to 6 classes.
Found 3000 files belonging to 6 classes.
Found 3000 files belonging to 6 classes.


displaying the shape of the data and labels

In [38]:
for data_batch, labels_batch in train_dataset:
    print("Data batch shape: ", data_batch.shape)

    print("Labels batch shape: ", labels_batch.shape)
    break

Data batch shape:  (32, 180, 180, 3)
Labels batch shape:  (32,)


In [39]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="../models/checkpoints/intel_image_clf.keras",
        save_best_only=True,
        monitor='val_accuracy'
    )
]

In [40]:
history = model.fit(
    train_dataset,
    epochs=2,
    validation_data=validation_dataset,
    callbacks=callbacks
)

Epoch 1/2


/home/avizyt/anaconda3/envs/tflow/lib/python3.9/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)
2022-04-08 22:56:05.992795: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 129777664 exceeds 10% of free system memory.
2022-04-08 22:56:06.317993: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 32444416 exceeds 10% of free system memory.
2022-04-08 22:56:06.337519: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 62005248 exceeds 10% of free system memory.
2022-04-08 22:56:06.440146: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 27541504 exceeds 10% of free system memory.
2022-04-08 22:56:06.758401: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Alloc

188/188 [==============================] - 267s 1s/step - loss: 1.1574 - accuracy: 0.5373 - val_loss: 0.9072 - val_accuracy: 0.6377
Epoch 2/2
188/188 [==============================] - 264s 1s/step - loss: 0.8617 - accuracy: 0.6703 - val_loss: 0.8003 - val_accuracy: 0.6890


In [41]:
test_model = keras.models.load_model("../models/checkpoints/intel_image_clf.keras")
test_loss, test_acc = test_model.evaluate(test_dataset)
print(f"Test Loss: {test_loss},Test Accuracy: {test_acc}")

/home/avizyt/anaconda3/envs/tflow/lib/python3.9/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


94/94 [==============================] - 31s 314ms/step - loss: 0.8456 - accuracy: 0.6703
Test Loss: 0.8456147313117981,Test Accuracy: 0.6703333258628845
